In [50]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from  torch.nn.functional import one_hot
import h5py
from sklearn.utils import shuffle
import sys
import datetime as dt

device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

dtype = torch.double
   
# Get functions from other notebooks
%run /tigress/kendrab/analysis-notebooks/preproc_utils.ipynb
%run /tigress/kendrab/analysis-notebooks/eval_utils.ipynb

Using cuda device


In [51]:
model_file = "/tigress/kendrab/analysis-notebooks/model_outs/01-08-23/samples/A131937_modelfile.tar"

### Paste model parameters here

In [52]:
model_name = "A"

# hyperparameters
padding_length = 10  # amount of data on each side of each segment for additional info
stride = 10  # size (and therefore spacing) of each segment
input_length = stride + 2*padding_length
kernel_size = 3
pool_size = 2
out_channels = 32  # like 'filters' in keras
thinning_factor = [0.9, None]
learning_rate = 0.01
epochs = 15
hyperparams = {'learning_rate':learning_rate, 'out_channels':out_channels, 'kernel_size':kernel_size, 'pool_size':pool_size,
              'input_length':input_length, 'stride':stride, 'epochs':epochs, 'thinning_factor':thinning_factor}

# other parameters
batch_size = 256  # idk what this should be for best performance 

### Paste model class here

In [53]:
class ModelA(nn.Module):
    """ 1D CNN Model """
    def __init__(self):
        super().__init__()
        # define these all separately because they will get different weights
        # consider smooshing these together into one convolution with in_channels=5
        self.bx_layers = nn.Sequential(nn.Conv1d(1, out_channels, kernel_size, padding='valid'),
                                       nn.ReLU(),
                                       nn.MaxPool1d(pool_size))
        self.by_layers = nn.Sequential(nn.Conv1d(1, out_channels, kernel_size, padding='valid'),
                                       nn.ReLU(),
                                       nn.MaxPool1d(pool_size))
        self.bz_layers = nn.Sequential(nn.Conv1d(1, out_channels, kernel_size, padding='valid'),
                                       nn.ReLU(),
                                       nn.MaxPool1d(pool_size))
        self.jy_layers = nn.Sequential(nn.Conv1d(1, out_channels, kernel_size, padding='valid'),
                                       nn.ReLU(),
                                       nn.MaxPool1d(pool_size))
        self.vz_layers = nn.Sequential(nn.Conv1d(1, out_channels, kernel_size, padding='valid'),
                                       nn.ReLU(),
                                       nn.MaxPool1d(pool_size))
        # TODO split this into CNN and classifier parts to facilitate domain adaptation
        self.post_merge_layers = nn.Sequential(nn.Conv1d(out_channels, out_channels*2, kernel_size,
                                                         padding='valid'),
                                               nn.ReLU(),
                                               nn.MaxPool1d(pool_size),
                                               nn.Flatten(),
                                               nn.LazyLinear(stride*2),
                                               nn.ReLU(),
                                               nn.Unflatten(1,(2,stride)))
                                               

    def forward(self, bx, by, bz, jy, vz):
        bx_proc = self.bx_layers(bx)
        by_proc = self.by_layers(by)
        bz_proc = self.bz_layers(bz)
        jy_proc = self.jy_layers(jy)
        vz_proc = self.vz_layers(vz)
        combined = .2*(bx_proc + by_proc + bz_proc + jy_proc + vz_proc)
        logits = self.post_merge_layers(combined)
        
        return logits

In [54]:
def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)  # number of samples
    tot_points = size*stride
    num_batches = len(dataloader)
    test_loss_sum, correct = 0, 0

    with torch.no_grad():
        for _, _, bx, by, bz, jy, vz, _, _, y in dataloader:
            pred = model(bx, by, bz, jy, vz)
            test_loss_sum += loss_fn(pred, y).item()  # .item() fetches the python scalar
            # number of correct per-point predictions
            correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
    ##### FIX VECTOR LOSS STUFF
    ##### WHAT SHAPE SHOULD LOSS BE? FIGURE IT OUT AND MAKE IT SO
    ##### THEN HAVE THE DIAGNOSTICS CALCULATED CORRECTLY
    test_loss_sum /= num_batches
    correct /= tot_points
    print(f"Test Error: \n Accuracy: {(100*correct):>0.5f}%, Avg loss: {test_loss_sum:>8f} \n")

### Restore the model

In [55]:
model = ModelA()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

checkpoint = torch.load(model_file)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss_fn = checkpoint['loss_fn']

model.eval()  # set to correct mode to get the correct results

ModelA(
  (bx_layers): Sequential(
    (0): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=valid)
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (by_layers): Sequential(
    (0): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=valid)
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (bz_layers): Sequential(
    (0): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=valid)
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (jy_layers): Sequential(
    (0): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=valid)
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (vz_layers): Sequential(
    (0): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=valid)
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=Fal

### Load and preprocess the data to feed into the model

In [62]:
# TODO use command line args or someting easier than throwing it here
basedir = '/tigress/kendrab/21032023/'
readpaths = []

for i in range(10):
    totdir = basedir+str(i)+'/'
    for j in range(5,60,5):
        readpaths.append(totdir+f"100samples_idx{j}_bxbybzjyvzexeyez.hdf5")
        
idx_list = []  # to keep track of which file what sample came from
s_list = []
bx_list = []
by_list = []
bz_list = []
jy_list = []
vz_list = []
x0_list = []
x1_list = []
topo_list = []

train_idx = None

for idx, filepath in enumerate(readpaths):
    with h5py.File(filepath, 'r') as file:
        print(file.keys())
        print(file['x_mms'][0])
        sys.exit()
        idx_list += [np.array([idx for i in bx]) for bx in file['bx_smooth'][:]]  # check this structure!!!
        s_list += list(file['s'][:])
        bx_list += list(file['bx_smooth'][:])
        by_list += list(file['by'][:])
        bz_list += list(file['bz_smooth'][:])
        jy_list += list(file['jy'][:])
        vz_list += list(file['vz'][:]) 
        x0_list += list(file['x_mms'][:])
        x1_list += list(file['z_mms'][:])
        topo_list_tmp = list(file['topo'][:])
        for i in range(len(topo_list_tmp)):  # I tried to vectorize this but I didn't get it to work
            topo_list_tmp[i] = torch.from_numpy(topo_list_tmp[i].astype(int) % 2)  # cat 0,2 are not plasmoids, cat 1,3 are
            topo_list_tmp[i] = one_hot(topo_list_tmp[i], num_classes=2)
        topo_list += topo_list_tmp
        
        if idx == int(.7*len(readpaths)):  # roughly 70-30 train-test split for now
            train_idx = len(bx_list)

print(len(bx_list))
# do train test split
idx_train_list = idx_list[:train_idx]  # to keep track of which file what sample came from
s_train_list = s_list[:train_idx] 
bx_train_list = bx_list[:train_idx] 
by_train_list = by_list[:train_idx] 
bz_train_list = bz_list[:train_idx] 
jy_train_list = jy_list[:train_idx] 
vz_train_list = vz_list[:train_idx] 
x0_train_list = x0_list[:train_idx] 
x1_train_list = x1_list[:train_idx] 
topo_train_list = topo_list[:train_idx] 

idx_test_list = idx_list[train_idx:] 
s_test_list = s_list[train_idx:] 
bx_test_list = bx_list[train_idx:] 
by_test_list = by_list[train_idx:] 
bz_test_list = bz_list[train_idx:] 
jy_test_list = jy_list[train_idx:] 
vz_test_list = vz_list[train_idx:] 
x0_test_list = x0_list[train_idx:] 
x1_test_list = x1_list[train_idx:] 
topo_test_list = topo_list[train_idx:] 

# BUT WAIT THERE'S MORE! Include the slices from plain ol current sheets. Split 50-50 between train and test
# lots of magic numbers here but we don't have time to make the code nice rn
noplasmoids_dir = '/tigress/kendrab/06022023/'
noplasmoids_paths = []
for j in range(5,55,5):
        noplasmoids_paths.append(noplasmoids_dir+f"100samples_idx{j}_bxbybzjyvzexeyez.hdf5")
        
for k in range(5):
    # training part
    with h5py.File(noplasmoids_paths[k], 'r') as file:
        idx_train_list += [np.array([idx for i in bx]) for bx in file['bx_smooth'][:]]  # check this structure!!!
        s_train_list += list(file['s'][:])
        bx_train_list += list(file['bx_smooth'][:])
        by_train_list += list(file['by'][:])
        bz_train_list += list(file['bz_smooth'][:])
        jy_train_list += list(file['jy'][:])
        vz_train_list += list(file['vz'][:]) 
        x0_train_list += list(file['x_mms'][:])
        x1_train_list += list(file['z_mms'][:])
        topo_list_tmp = list(file['topo'][:])
        for i in range(len(topo_list_tmp)):  # I tried to vectorize this but I didn't get it to work
            topo_list_tmp[i] = torch.from_numpy(topo_list_tmp[i].astype(int) % 2)  # cat 0,2 are not plasmoids, cat 1,3 are
            topo_list_tmp[i] = one_hot(topo_list_tmp[i], num_classes=2)
        topo_train_list += topo_list_tmp    
        
    # testing part
    with h5py.File(noplasmoids_paths[k+5], 'r') as file:
        idx_test_list += [np.array([idx for i in bx]) for bx in file['bx_smooth'][:]]  # check this structure!!!
        s_test_list += list(file['s'][:])
        bx_test_list += list(file['bx_smooth'][:])
        by_test_list += list(file['by'][:])
        bz_test_list += list(file['bz_smooth'][:])
        jy_test_list += list(file['jy'][:])
        vz_test_list += list(file['vz'][:]) 
        x0_test_list += list(file['x_mms'][:])
        x1_test_list += list(file['z_mms'][:])
        topo_list_tmp = list(file['topo'][:])
        for i in range(len(topo_list_tmp)):  # I tried to vectorize this but I didn't get it to work
            topo_list_tmp[i] = torch.from_numpy(topo_list_tmp[i].astype(int) % 2)  # cat 0,2 are not plasmoids, cat 1,3 are
            topo_list_tmp[i] = one_hot(topo_list_tmp[i], num_classes=2)
        topo_test_list += topo_list_tmp        

<KeysViewHDF5 ['bx', 'bx_mms', 'bx_smooth', 'by', 'by_mms', 'bz', 'bz_mms', 'bz_smooth', 'current_sheets', 'd_per_de', 'ex', 'ex_mms', 'ey', 'ey_mms', 'ez', 'ez_mms', 'flux_fn', 'fluxfn_hessian_det', 'jy', 'jy_mms', 'o_coords', 'o_structures', 's', 'separatrices', 'topo', 'unit_vec', 'vx_mms', 'vz', 'x_coords', 'x_mms', 'z_mms', 'zero_pt']>
[-208.41887901 -208.1949515  -207.97102398 -207.74709647 -207.52316895
 -207.29924143 -207.07531392 -206.8513864  -206.62745889 -206.40353137
 -206.17960386 -205.95567634 -205.73174883 -205.50782131 -205.2838938
 -205.05996628 -204.83603876 -204.61211125 -204.38818373 -204.16425622
 -203.9403287  -203.71640119 -203.49247367 -203.26854616 -203.04461864
 -202.82069113 -202.59676361 -202.37283609 -202.14890858 -201.92498106
 -201.70105355 -201.47712603 -201.25319852 -201.029271   -200.80534349
 -200.58141597 -200.35748846 -200.13356094 -199.90963342 -199.68570591
 -199.46177839 -199.23785088 -199.01392336 -198.78999585 -198.56606833
 -198.34214082 -198

SystemExit: 

### Train test split

In [ ]:
# chunk into sliding windows
# NOTE TOPO HAS DIFFERENT SEGMENT LENGTHS THAN THE INPUTS (stride vs. 2*padding+stride)
idx_train = batch_unpadded_subsects(idx_train_list, padding_length, stride)
s_train = batch_subsects(s_train_list, input_length, stride)  # not going through training so don't need to shape right
bx_train = np.expand_dims(batch_subsects(bx_train_list, input_length, stride),1)
by_train = np.expand_dims(batch_subsects(by_train_list, input_length, stride),1)
bz_train = np.expand_dims(batch_subsects(bz_train_list, input_length, stride),1)
jy_train = np.expand_dims(batch_subsects(jy_train_list, input_length, stride),1)
vz_train = np.expand_dims(batch_subsects(vz_train_list, input_length, stride),1)
x0_train = batch_unpadded_subsects(x0_train_list, padding_length, stride)
x1_train = batch_unpadded_subsects(x1_train_list, padding_length, stride)
topo_train = np.swapaxes(batch_unpadded_subsects(topo_train_list, padding_length, stride), 1, 2)

print(bx_train.shape)

idx_test = batch_unpadded_subsects(idx_test_list, padding_length, stride)
s_test = np.expand_dims(batch_subsects(s_test_list, input_length, stride),1)
bx_test = np.expand_dims(batch_subsects(bx_test_list, input_length, stride),1)
by_test = np.expand_dims(batch_subsects(by_test_list, input_length, stride),1)
bz_test = np.expand_dims(batch_subsects(bz_test_list, input_length, stride),1)
jy_test = np.expand_dims(batch_subsects(jy_test_list, input_length, stride),1)
vz_test = np.expand_dims(batch_subsects(vz_test_list, input_length, stride),1)
x0_test = batch_unpadded_subsects(x0_test_list, padding_length, stride)
x1_test = batch_unpadded_subsects(x1_test_list, padding_length, stride)
topo_test = np.swapaxes(batch_unpadded_subsects(topo_test_list, padding_length, stride), 1, 2)

# shuffle the segments so they aren't adjacent to overlapping/similar segments
idx_train, s_train, bx_train, by_train, bz_train, jy_train, vz_train, x0_train, x1_train, topo_train = \
    shuffle(idx_train, s_train, bx_train, by_train, bz_train, jy_train, vz_train, x0_train, x1_train, topo_train)

idx_test, s_test, bx_test, by_test, bz_test, jy_test, vz_test, x0_test, x1_test, topo_test = \
    shuffle(idx_test, s_test, bx_test, by_test, bz_test, jy_test, vz_test, x0_test, x1_test, topo_test)

# try to do some rebalancing in the training set
# model is struggling on plasmoids, which are underrepresented
[idx_train, s_train, bx_train, by_train, bz_train, jy_train, vz_train, x0_train, x1_train], topo_train = \
    rebalance_ctrl_group([idx_train, s_train, bx_train, by_train, bz_train, jy_train, vz_train, x0_train, x1_train],
                         topo_train, null_label=[1,0], thinning_factor = thinning_factor[0])
# numpy arrays to torch tensors (while crying about how many lines of code this is surely there is a better way)
idx_train = torch.from_numpy(idx_train).to(device, dtype=dtype)
s_train = torch.from_numpy(s_train).to(device, dtype=dtype)
bx_train = torch.from_numpy(bx_train).to(device, dtype=dtype)
by_train = torch.from_numpy(by_train).to(device, dtype=dtype)
bz_train = torch.from_numpy(bz_train).to(device, dtype=dtype)
jy_train = torch.from_numpy(jy_train).to(device, dtype=dtype)
vz_train = torch.from_numpy(vz_train).to(device, dtype=dtype)
x0_train = torch.from_numpy(x0_train).to(device, dtype=dtype)
x1_train = torch.from_numpy(x1_train).to(device, dtype=dtype)
topo_train = torch.from_numpy(topo_train).to(device, dtype=dtype)

idx_test = torch.from_numpy(idx_test).to(device, dtype=dtype)
s_test = torch.from_numpy(s_test).to(device, dtype=dtype)
bx_test = torch.from_numpy(bx_test).to(device, dtype=dtype)
by_test = torch.from_numpy(by_test).to(device, dtype=dtype)
bz_test = torch.from_numpy(bz_test).to(device, dtype=dtype)
jy_test = torch.from_numpy(jy_test).to(device, dtype=dtype)
vz_test = torch.from_numpy(vz_test).to(device, dtype=dtype)
x0_test = torch.from_numpy(x0_test).to(device, dtype=dtype)
x1_test = torch.from_numpy(x1_test).to(device, dtype=dtype)
topo_test = torch.from_numpy(topo_test).to(device, dtype=dtype)

# collect data into Datasets
train_dset = TensorDataset(idx_train, s_train, bx_train, by_train, bz_train, jy_train, vz_train,
                              x0_train, x1_train, topo_train)
test_dset =  TensorDataset(idx_test, s_test, bx_test, by_test, bz_test, jy_test, vz_test,
                              x0_test, x1_test, topo_test)
# Make DataLoaders for the training and test data
train_dl = DataLoader(train_dset, batch_size = batch_size)
test_dl = DataLoader(test_dset, batch_size = batch_size)

### Do whatever testing you want with the model